# Example Fit

This example shows a complete analysis of the data set included in Exercise 6.2 of [*Measurements and their Uncertainties*](https://www.oupcanada.com/catalog/9780199566334.html), by Hughes and Hase.

* [Preliminaries](#Preliminaries)
* [Step 1: Import packages](#Step-1:-Import-packages)
* [Step 2: Load the data](#Step-2:-Load-the-data)
    * [Examine the format](#Examine-the-format)
    * [Import the data into an array](#Import-the-data-into-an-array)
* [Step 3: Plot the data](#Step-3:-Plot-the-data)
* [Step 4: Define the model function](#Step-4:-Define-the-model-function)
    * [Tip: Express the model function in appropriate units](#Tip:-Express-the-model-function-in-appropriate-units)
* [Step 5: Choose initial parameter values for the model](#Step-5:-Choose-initial-parameter-values-for-the-model)
* [Step 6: Fit the model to the data](#Step-6:-Fit-the-model-to-the-data)
* [Step 7: Assess the fit](#Step-7:-Assess-the-fit)
    * [Visual inspection](#Visual-inspection)
    * [Compute the $\chi^2$ statistic](#Compute-the-$\chi^2$-statistic)
    * [Plot the (normalized) residuals](#Plot-the-(normalized)-residuals)
* [Step 8: Decide what to do next](#Step-8:-Decide-what-to-do-next)
* [Summary](#Summary)

## Preliminaries
This analysis is done in the Python programming language using a Jupyter notebook interface. Python has hundreds of specialized packages for scientific applications, but we will restrict ourselves here to the core packages that are commonly used in data analysis. We will describe them as we use them, leaving a more comprehensive introduction to Python and its associated scientific programming ecosystem for later. You can find more documentation at the links below.
* [Python](https://docs.python.org/3/tutorial/index.html): the high-level computing language
* [Jupyter](https://jupyter-notebook.readthedocs.io/en/latest/): the notebook interface for Python programming and documentation
* [NumPy](https://numpy.org/doc/stable/user/index.html): a numerical computing package
* [matplotlib](https://matplotlib.org/stable/users/index.html): a scientific visualization package
* [SciPy](https://docs.scipy.org/doc/scipy/tutorial/index.html): an extension of the NumPy package that includes additional functionality for scientific applications
* [Markdown](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html): the markup language used for formatting text cells in Jupyter—this is not technically part of Python, but is useful for documenting Python-based analysis in Jupyter notebooks

## Step 1: Import packages
We will need NumPy, the `pyplot` subpackage from matplotlib, and the `curve_fit` routine from SciPy, so we import these into the notebook in the following cell.

The first line in the cell is a comment, `# Import packages`, which we identify by beginning the line with the `#` character. Python ignores these lines, so we can use them to include brief comments in plain English that describe what the Python code does.

The first line of code in the cell tells Python to load the `numpy` package under the name `np`, which is a standard abbreviation that is more convenient for typing. Similarly, `matplotlib.pyplot` is commonly abbreviated as `plt`. Both `np` and `plt` include a variety of routines, as we will see below. The third line imports just the `curve_fit` routine from the `scipy.optimize` subpackage of SciPy, without assigning an abbreviated name to it. Similarly, the fourth line imports just the `chi2` routine from the `scipy.stats` subpackage of SciPy.

The last line, `%matplotlib inline`, is preceded by a `%` to signal that it is not a Python command but a system  command, known as a *magic command*. In this case the system is the Jupter notebook server, and `%matplotlib inline` tells it to show plots from Matplotlib in the notebook, or *inline*.

In [ ]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import chi2

# Tell notebook server to show plots inline
%matplotlib inline

## Step 2: Load the data

### Examine the format
The data for this example is contained in the file `Example-Data.csv`, which you should open in a text editor to view. But since we are already in a Jupyter notebook, we can also use Python built-in functions to view its contents. In the cell below, the first line opens the file, and the second line displays its contents.

In [ ]:
# Display file contents
with open("data/Example-Data.csv") as file:
    print(file.read())

The file contains three columns of data in what is known as *comma-separated value* or CSV format. Each line includes three items, separated from each other by commas—thus the name "comma-separated value." The first line includes the labels for each column, and all subsequent lines represent measurements of the frequency, voltage, and voltage error. For simple tabular data this is the best format to use, since it is platform-independent, human-readable, and widely adopted by other software packages, including NumPy and Microsoft Excel. The SFU Library maintains a [list of recommended file formats](https://www.lib.sfu.ca/help/publish/research-data-management/recommended-file-formats) that are appropriate for research data.

### Import the data into an array
So far, all we have done is displayed the contents of `Example-Data.csv` on the screen. For analysis, we need to convert the file into an array of *numbers*, which we will do using the `genfromtext` routine from NumPy.

In the cell below, the call to `np.genfromtext` in the first line tells Python to find the `genfromtext` routine in the NumPy package (loaded at the beginning under the label `np`), then use it to convert the contents of `Example-Data.csv` into a numerical array that we assign to `data`. The call to `genfromtext` includes three arguments: the file name, the delimeter (i.e., the character that separates items in different columns), and the number of header lines to skip. The `genfromtext` [help](https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html) page describes many more options that can be included, depending on the circumstances. The `print` functions in the second and third lines show the contents of this array.

In [ ]:
# Load file into array
data = np.genfromtxt("data/Example-Data.csv", delimiter=",", skip_header=1)
print("data =")
print(data)

Let's take a moment to compare the output of the `print(data)` statement above with that of the `print(file.read())` statement in the previous cell. Whereas the `print(file.read())` statement simply copied the text contained in `Example-Data.csv` to the screen, the `print(data)` statement *converts* the numerical information in `data` back into a standardized text format for display. Each row of the array is indicated by square brackets (`[]`), and these rows are then stacked into a larger array that is also enclosed by square brackets. The overall output, then, reads as follows:

1. The `[[` at the beginning tells us we have a two-dimensional array. The first `[` denotes the start of the entire set of rows, and the second `[` denotes the start of the first row.
2. The first row is `[ 10.  16.   5.]`, the second row is `[ 20.  45.   5.]`, etc.
3. The `]]` at the end tells us that we have come to the end of the two-dimensional array. The first `]` denotes the end of the last row, and the second `]` denotes the end of the entire set of rows.

Each element of `data` is associated with a set of *indices*, which we can access using the notation `data[i,j]`, where `i` is the row number and `j` is the column number. Indices start from zero in Python, so in this example, the eleven rows and three columns of `data` are indexed by `i = 0, 1, ... 10` and `j = 0, 1, 2`.

To check that you understand how indexing works, compare the array above with the output from the cell below.

In [ ]:
# Display some array elements
first = data[0, 0]
last = data[10, 2]
middle = data[5, 1]
print("first =", first)
print("last =", last)
print("middle =", middle)

The colon character `:` is a shorthand notation for the entire set of values allowed for a particular index, and allows us to access to an entire row or column. In the cell below, we use it to assign each column of `data` to the new variables `frequency`, `voltage`, and `err`, for later use.

In [ ]:
# Assign each array column to a separate array
frequency = data[:, 0]
voltage = data[:, 1]
err = data[:, 2]
print("frequency =", frequency)
print("voltage =", voltage)
print("err =", err)

## Step 3: Plot the data
To plot the data for analysis, we turn to the routines contained in `matplotlib.pyplot`, which we imported earlier under the abbreviated name `plt`. All the PyPlot routines are listed on its [help page](https://matplotlib.org/stable/api/pyplot_summary.html), which links to additional help on each routine.

The following cell shows the voltage as a function of frequency, with the voltage uncertainty represented by an error bar. The first line calls the `errorbar` routine from `plt`. The first two arguments of `errorbar` are required, and correspond to the $x$ and $y$ variables in the plot, respectively. The third argument, `yerr=err`, tells `errorbar` to show errobars that extend `err` above and below the data point along the $y$ axis. The fourth argument, `fmt='o'` causes the data points to be represented as circles (`o`). For more details on `errorbar`, see its [help page](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.errorbar.html). After the `errorbar` statement, the next three lines produce labels for the $x$ and $y$ axes, along with the title. The last line displays the plot.

In [ ]:
# Plot data
plt.errorbar(frequency, voltage, yerr=err, fmt="o")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Voltage (mV)")
plt.title("Data from Exercise 6.2")
plt.show()

## Step 4: Define the model function
In this example we fit the data with a simple linear model,

$$y = mx+b,$$

where $x$ represents the frequency, $y$ the voltage, and $m$ and $b$ represent *unknown fit parameters* that we will determine from the data.

In the cell below we use the Python statement `def` to define this model function. The first line specifies `model` as the name of the function and indicates what arguments it expects. For consistency with the `curve_fit` routine, we reserve the first argument for the independent variable `x`, followed by the fit parameters `m` and `b`. The second line uses the Python statement `return` to specify the output of `model`, which in this case is the output of the expression `m * x + b`. *It is important to note here that the* `return` *statment is indented.* Unlike many other programming languages, Python assigns significance to indentation. Here, it assumes that any indented text after the `def` statement is part of the function definition.

### Tip: Express the model function in appropriate units
The default behavior of `curve_fit` is to stop adjusting any fit parameter that has been specified to an absolute numerical precision of 10<sup>-8</sup> or better. This will cause your fit to stop prematurely if you use units that make your optimal fit parameters numerically small, so you should typically use units for both your data and your fit parameters that yield numerical values much larger than 10<sup>-8</sup>. For example, express the mercury green emission wavelength as 546.1 nm or 0.5461 µm, not 5.461 &times; 10<sup>-7</sup> m. In this example, the data are expressed in millivolts and hertz, which yields numerical values for both the data and the optimal fit parameters that are numerically much larger than 10<sup>-8</sup>.

In [ ]:
# Define model function


def model(x, m, b):
    return m * x + b

## Step 5: Choose initial parameter values for the model
After defining the model, it is generally good practice to compare the model to the data at a qualitative level before handing it off to the computer for a quantitative fit. Doing this will help you catch any mistakes in your model definition, develop an intuition for how the fit quality varies with the parameters, and choose initial values for the parameters that are reasonably close to optimal. The simple model in our example will converge regardless of the initial parameter choice, but in more complex models, having good values for the initial fit parameters is often essential to achieving a good fit.

If you inspect the raw data in the errorbar plot above, you should see that the slope is close to two and the intercept is close to zero. You can also see this relationship in the tabular data, where most of the voltage values (in Hz) are about twice the frequency values (in mV). Consequently, we assign the initial parameter values `m_init = 2` and `b_init = 0` in the first two lines of code in the cell below. The remaining lines create a plot that shows how the model fits the data with these preliminary values for the fit parameters.

First, we use the NumPy routine `linspace` to create `f_model`, an array with 1000 equally-spaced points ranging from 0 to 120. This allows us to extend the model curve to frequencies outside the range of the original data. If necessary, we can adjust this point density to ensure there are no visual differences between the plotted function and the continuous curve it is representing.

Next, we use `plot` from PyPlot to show the model curve evaluated at all fifty points in `f_model` with the parameters `m_init` and `b_init`. The third argument, `'r-'`, tells `plot` to represent this as a red line. The next lines use `errorbar`, `xlabel`, `ylabel`, and `title` to add the data as an errorbar plot, label the axes, and add a title, just as before. We then use `xlim` and `ylim` to adjust the limits of the $x$ and $y$ axes, respectively, and `show` to display the final plot.

In [ ]:
# Set initial parameters m0 and b0
m_init = 2
b_init = 0

# Define new frequency array, f_model, for displaying the model
f_model = np.linspace(0, 120, 1000)

# Make the plot
# Recall the order of the arguments in the model definition, model(x, m, b):
plt.plot(f_model, model(f_model, m_init, b_init), "r-")
plt.errorbar(frequency, voltage, yerr=err, fmt="o")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Voltage (mV)")
plt.title("Data with linear model, initial parameters")
plt.xlim(0, 120)
plt.ylim(0, 250)
plt.show()

This is already good agreement, but we can do better, as we will see in the next step.

## Step 6: Fit the model to the data
Now we have everything we need to use `curve_fit` to find the parameter values that provide the best fit of the model to the data. As described in the [help](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html), `curve_fit` has three required arguments: the model, the data $x$ values, and the data $y$ values, in that order. We also include three additional optional arguments. The expression `p0=[m_init, b_init]` specifies the initial guess for the parameters, expressed as a single array in the order of their appearance in `model`. Next, `sigma=err` specifies the uncertainty in the data $y$ values, and `absolute_sigma=True` indicates that these uncertainties are known independently, as is usually the case in physics but less common in other disciplines.

The first output of `curve_fit`, `p_opt`, is an array of the optimal parameter values, listed again in the order of their appearance in `model`. We can write it in matrix form as

$$ \hat{\mathbf{p}} = \begin{bmatrix}\hat{p}_0\\ \hat{p}_1\end{bmatrix} = \begin{bmatrix}\hat{m}\\ \hat{b}\end{bmatrix},$$

where the notation $\hat{m}$ indicates the optimal value for the parameter $m$.

The second output, `p_cov`, is a two-dimensional array that corresponds to the parameter *covariance matrix*, which provides an estimate of the parameter uncertainty and correlation. As described in Chapter 7 of Hughes and Hase, the diagonal elements of `p_cov` correspond to the *variance* of the associated parameters. In our example, the covariance matrix is

$$ \mathbf{C} = \begin{bmatrix}C_{00} & C_{01}\\ C_{10} & C_{11}\end{bmatrix} = \begin{bmatrix}\alpha_m^2 & \rho_{mb}\alpha_m\alpha_b\\ \rho_{mb}\alpha_m\alpha_b & \alpha_b^2\end{bmatrix}, $$

where $\alpha_m$ is the standard error in the slope estimate $\hat{m}$, $\alpha_b$ is the standard error in the intercept estimate $\hat{b}$, and $\rho_{mb}$ is the *correlation coefficient* between $\hat{m}$ and $\hat{b}$, discussed in Sec. 7.3 of Hughes and Hase.

In [ ]:
# Fit the model to the data and display results
# The order of the parameter array elements p0, p_opt, and p_cov
# are determined by their order [m,b] in the model definition, model(x,m,b):
p_opt, p_cov = curve_fit(model, frequency, voltage, p0=[m_init, b_init], sigma=err, absolute_sigma=True)
print("[m_opt b_opt] =", p_opt)
print()
print("p_cov =")
print(p_cov)

Now we can assign the results of `curve_fit` to new variables that are easier to work with. In the cell below, we assign the first element of `p_opt` to `m_opt`, and the second element to `b_opt`.

Next, we use the definition of $\mathbf{C}$ above to compute $\alpha_m$, $\alpha_b$, and $\rho_{mb}$. We denote the estimated standard error in `m_opt` as `m_alpha`, given by $\alpha_m = \sqrt{C_{00}}$. Similarly, we donote the standard error in `b_opt` as `b_alpha`, given by $\alpha_b = \sqrt{C_{11}}$. Finally, we denote the correlation coefficient between `m_opt` and `b_opt` as `rho_mb`, given by $\rho_{mb} = \sqrt{C_{01}}/(\alpha_m\alpha_b)$.

In [ ]:
# Assign results of curve_fit to new variables
m_opt = p_opt[0]
b_opt = p_opt[1]
m_alpha = np.sqrt(p_cov[0, 0])
b_alpha = np.sqrt(p_cov[1, 1])
rho_mb = p_cov[0, 1] / (m_alpha * b_alpha)

The cell below uses a series of `print` statements to display these variables, using the default precision. 

In [ ]:
# Display formatted results
print("Default precision:")
print("Model slope (mV/Hz):    ", m_opt, " ± ", m_alpha)
print("Model intercept (mV):", b_opt, " ± ", b_alpha)
print("Correlation coefficient:", rho_mb)

_**Note that you should always round such parameter estimates and uncertainties to the appropriate number of significant figures!**_

Here is you how you *should* report these numbers:

```
Corrected precision:
Model slope (mV/Hz): 2.03  ±  0.05
Model intercept (mV): -1  ±  3
Correlation coefficient: -0.9
```

These estimates follow the "golden rules" described in Sec. 2.9 of Hughes and Hase, adapted to the model-fitting context.

1. *If and only if the fit between a model and the data is good* (see below for methods to assess fit quality), the best estimate of a parameter is the one returned by the fit, and may be understood as the mean value of the distribution that you would get if you were to conduct the same experiment many times.
2. The error in each parameter is given by the square root of the associated diagonal element of the covariance matrix, and may be understood as the standard error for the parameter. Correlation coefficients between parameters may also be derived from the covariance matrix, using Eq. (7.30) in Hughes and Hase.
3. Round up the error for each parameter to the appropriate number of significant figures (usually just one, but sometimes more—see the discussion on p. 17 of Hughes and Hase, including Footnote #7).
4. Match the number of decimal places in the mean to the standard error.
5. Include units.

The cell below uses Python [f-strings](https://docs.python.org/3/reference/lexical_analysis.html#f-strings) to display the variables with the correct precision. For example, `{m_opt:.2f}` in the first line indicates that `m_opt` should be displayed as a floating point number to two digits of precision (`.2f`). Later in the same line, `{m_alpha:.2f}` indicates that `m_alpha` should be displayed with the same precision. See [here](https://docs.python.org/3/tutorial/inputoutput.html#formatted-string-literals) for additional examples.

In [ ]:
# Display formatted results at correct level of significance
# Adjust each format string as appropriate
print("Correctly formatted precision:")
print(f"Model slope (mV/Hz):     {m_opt:.2f} ± {m_alpha:.2f}")
print(f"Model intercept (mV):    {b_opt:.0f} ± {b_alpha:.0f}")
print(f"Correlation coefficient: {rho_mb:.1f}")

## Step 7: Assess the fit
The fit parameters and uncertainties returned by `curve_fit` are only as good as the fit itself, so it is *essential* that you evaluate the fit quality before relying on them. Hughes and Hase describe several ways to do this, and we will show below how to use Python for the most common ones.

### Visual inspection
We already compared the model to the data for the initial parameters, so we can use the same routines to do the same comparison with the optimal parameters.

In [ ]:
# Plot data with best-fit model
plt.plot(f_model, model(f_model, m_opt, b_opt), "r-")
plt.errorbar(frequency, voltage, yerr=err, fmt="o")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Voltage (mV)")
plt.title("Data with linear model, optimal parameters")
plt.xlim(0, 120)
plt.ylim(0, 250)
plt.show()

### Compute the $\chi^2$ statistic
Beyond visual inspection, also known perjoratively as *$\chi$-by-eye*, we can assess the fit more quantitatively by computing the $\chi^2$ statistic,

$$ \chi^2_\text{min} = \sum_{k=1}^N\frac{[y_k - f(x_k,\hat{\textbf{p}})]^2}{\alpha_k^2},$$

where $\hat{\textbf{p}}$ denotes the array of best-fit parameters, $f(x,\hat{\textbf{p}})$ is the model function with independent variable $x$ and parameter array $\hat{\textbf{p}}$, and the sum is over all $N$ data points $(x_k, y_k)$ and standard error $\alpha_k$ in the $y$ variable.

When the data is well described by the model, we expect $[y_k - f(x_k,\hat{\textbf{p}})]^2\approx \alpha_k^2$ for each $k$. We also have the freedom, however, to adjust a total of $m$ fit parameters to optimize the fit, so we expect $\chi^2_\text{min} \approx \nu$, where $\nu = N - m$ is the number of *statistical degrees of freedom*. This provides an objective way to test if a model is consistent with the data.

We compute this statistic for our example in the cell below, following three steps. First, we compute the fit *residuals*, given by the set $\{ y_k - f(x_k,\hat{\textbf{p}}) \},$ $k = 0, 1, ... N$, and assign it to the array `res`. Next, we normalize each residual value to its estimated uncertainty, and assign it to `normres`. Finally, we compute the $\chi^2$ statistic by summing over the squares of each normalized residual.

In [ ]:
# Compute and display chi-squared
res = voltage - model(frequency, m_opt, b_opt)
normres = res / err
chisq = np.sum(normres**2)
print(f"chisq = {chisq:.1f}")  # Adjust the format specification as appropriate

We have $N = 11$ and $m = 2$, so $\nu = 9$ and $\chi^2_\text{min} = 9.1\approx \nu$ suggests that the model is consistent with the data. We show how to determine $\nu$ with Python in the cell below, using the NumPy `size` function to determine the number of data points and parameters.

In [ ]:
# Compute and display DOF
n_data = np.size(voltage)
n_parm = np.size(p_opt)
dof = n_data - n_parm
print("dof =", dof)

To be even more quantitative, we can use the *cumulative distribution function* of the $\chi^2$ *distribution*, `chi2.cdf(chisq,dof)`, to determine the fraction of times that a fit with `dof` degrees of freedom will yield a value for $\chi_\text{min}^2$ less than or equal to `chisq`.

It is important to recognize that we are using the expression $\chi^2$ in two different ways here, as discussed in Secs. 8.2–8.6 of Hughes and Hase. The $\chi^2$ *statistic* is a *number* that we calculate from a fit to a given set of data, $\chi^2_\text{min}$. We expect this statistic to fluctuate probabilistically from one measurement to the next. The $\chi^2$ *distribution* is the probability distribution that we expect this statistic upon repeated measurements. It is a *mathematical function*, $\chi^2(x;\nu)$, that describes the probability that the statistic is equal to $x$ when there are $\nu$ statistical degrees of freedom. The cumulative distribution function $F$ may then be defined as

$$ F(x_1,\nu) = \text{Pr}(x<x_1,\nu) = \int_{0}^{x_1}\text{d}x\,\chi^2(x,\nu).$$

The following cell evaluates this function for $x_1 = \chi_\text{min}^2$.

In [ ]:
cdf = chi2.cdf(chisq, dof)
print(f"Cumulative probability = {cdf:.3f}")
print(f"Significance: {1-cdf:.3f}")

This tells us that we should expect to get $\chi^2_\text{min} = 9.1$ or less 57.3&nbsp;% of the time when the data is well-described by the model, and greater than this 42.7&nbsp;% of the time. We call this 42.7&nbsp;% probability the *significance* of the $\chi^2$ test for $\chi^2_\text{min} = 9.1$ and $\nu = 9$. The significance level may be understood as the probability of rejecting a fit erroneously, as a result of an unusually large statistical fluctuation in the data. A common convention is to reject any fit that falls below 5&nbsp;% significance. By this standard, we would accept this fit as consistent with the model.

### Plot the (normalized) residuals
Finally, we can check for statistical consistency by confirming that the residuals do not show any systematic pattern. In this example, the uncertainty in two of the measurements is much larger than the others, so the residual plot is overwhelmed by the deviations at these points, as seen below.

The following cell uses the PyPlot `stem` routine to produce the residual plot, which often shows deviations from the zero line more effectively than a regular scatterplot.

In [ ]:
# Show residuals
plt.stem(frequency, res)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Residual (mV)")
plt.title("Fit residuals")
plt.xlim(0, 120)
plt.show()

By plotting the normalized residuals, we can see that the large deviations at 50 Hz and 100 Hz are not unexpectedly large, and that the overall pattern of deviations is consistent with random fluctuations.

In [ ]:
# Show normalized residuals
plt.stem(frequency, normres)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Normalized Residual")
plt.title("Normalized residuals")
plt.xlim(0, 120)
plt.show()

## Step 8: Decide what to do next
Ideally you will get to the end of this whole process and find that the model fits the data well, and you can report your results with confidence. If time allows, you might even go back and take more data to improve the precision of your parameter estimates. But what happens if your $\chi^2$ statistic is too big or too small, or if the normalized residuals do not appear random? If that's the case, you have a few options:

* Look for defects in your experimental procedure, including the procedure you used to prepare the raw data for analysis (ie, incorrect unit conversion).
* Consider a different model. This may include changes that are based on physical reasoning, such as eliminating approximations to make it more realistic, or purely empirical, such as adding a constant background, or an extra term in a polynomial fit. The residuals should provide guidance on how to change the functional relationship to improve agreement. Just don't overdo it: if you find yourself needing to add many arbitrary fit parameters to achieve a good fit, that is usually a sign that you need to consider a different approach.
* Consider restricting your fit to a narrower range of measurements (i.e., focus on limiting values of $x$ and/or $y$, or on the heights, widths and locations of peaks instead of their detailed shape).
* Consider rescaling the uncertainties, as described in Sec. 8.9 of Hughes and Hase (though note their "health warning" in Footnote 9).
* Accept that you may be unable to remove all sources of systematic error in the given time, and do your best to make a quantitative estimate of your parameter uncertainties in light of them. You don't have to throw all of your data away just because you don't have a model that fits it well.


## Summary
This example can serve as a template for more general problems in data analysis, whether you use Python in a Jupyter notebook or some other computational tool. Let's review the procedure.

* *Load the data:* examine the data file format and import it into an array.
* *Plot the data:* visually inspect the relationship between the independent and dependent variables and evaluate the best way to model it.
* *Define the model:* the model should provide a plausible mathematical description of the data, and will usually involve unknown parameters that you will adjust to obtain the best fit.
* *Fit the model to the data:* pick good initial values for the unknown parameters, then use the computer to determine the optimal values and their uncertainties.
* *Assess the fit quality:* plot the model together with the data, evaluate the $\chi^2$ statistic, and inspect the residuals.
* *Decide what to do next:* if all goes well, report your results and move on. If not, use your results to guide changes to your experiment, your analysis, or both.

This example can also serve as a template for how to use Python in a Jupyter notebook to complete these steps. Let's review the basic Python concepts and expressions.

* *Importing packages:* these will typically include [NumPy](https://numpy.org/doc/stable/user/index.html), [matplotlib](https://matplotlib.org/users/index.html), and [SciPy](https://docs.scipy.org/doc/scipy/tutorial/index.html). See [here](https://docs.python.org/3/tutorial/modules.html#packages) for a general introduction to packages.
* *Importing data files:* for simple file formats, use [genfromtext](https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html).
* *Defining functions:* see [here](https://docs.python.org/3/tutorial/controlflow.html#defining-functions) for a general introduction to function definitions.
* *Plotting data:* this is what [matplotlib](https://matplotlib.org/users/index.html) was designed for. Within that package, [PyPlot](https://matplotlib.org/stable/api/pyplot_summary.html) has many routines for interactive plotting, such as [errorbar](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.errorbar.html), [plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html), and [stem](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.stem.html). Once you have the plot, you can use additional routines to format it, such as [xlabel](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xlabel.html), [ylabel](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.ylabel.html), [title](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.title.html), [xlim](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xlim.html), and [ylim](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.ylim.html).
* *Curve fitting:* the [curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) routine includes all the features that you need for basic use.
* *Mathematical functions:* Python can perform [basic mathematical operations](https://docs.python.org/3/tutorial/introduction.html#numbers), and NumPy includes several additional [mathematical functions](https://numpy.org/doc/stable/reference/routines.math.html) such as trigonometric functions, exponentials, array products and sums, etc.
* *The chi-squared distribution:* the [chi2](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html) routine may be used to evaluate fit quality.

##### About this notebook
© J. Steven Dodge, 2019. The notebook text is licensed under CC BY 4.0. See more at [Creative Commons](https://creativecommons.org/licenses/by/4.0/). The notebook code is open source under the [MIT License](https://opensource.org/licenses/MIT).